In [ ]:
import cv2#importing cv2 library
import numpy as np#importing numpy library

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
df=pd.read_excel("/content/drive/MyDrive/Video-Classification-GT.xlsx")

In [ ]:
df

,Video No.,Shot No.,Filename,Category,Unnamed: 4
0,1,1,Video001-Scene-001.mp4,Logo,NaN
1,1,2,Video001-Scene-002.mp4,Animation,NaN
2,1,3,Video001-Scene-003.mp4,IndoorLab,NaN
3,1,4,Video001-Scene-004.mp4,OutdoorGeneric,NaN
4,1,5,Video001-Scene-005.mp4,OutdoorLaunchpad,NaN
...,...,...,...,...,...
3606,25,51,Video025-Scene-051.mp4,Graphics,NaN
3607,25,52,Video025-Scene-052.mp4,Sky,NaN
3608,25,53,Video025-Scene-053.mp4,IndoorLab,NaN
3609,25,54,Video025-Scene-054.mp4,Animation,NaN


In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
#List- Normal Images
#List- Greyscale Images
#Canny Edges of Greyscale Iamges
#Consecutive Canny Edged Frames- Diff of Edges
#Mean, SD
#Threshold
#OG Frames- Mean, SD- If above threshold, we add to another list
#OUt of these list of frames, we chose the median of these frames as our most imp frame.

In [ ]:
def framing(video):#defining a small function named"framing" with a parameter "i" that's supposed to be provided for reading the video
    fr = []#creating an empty list named fr
    fr_pre=[]#creating an empty list named fr_pre
    cap = cv2.VideoCapture(video)#reading the video file
    while (cap.isOpened()):#This command builds a loop to check if the data is still being read from the video
        ret,frame = cap.read()#reading the data tunnel,gives two output where one tells about presence of frames(here it's ret) & the other speaks frame data(here it's frame)
        if ret == True:#checking for presence of frames
            # cv2_imshow(frame)#displaying the frames
            grayed = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)#Converting the frames to Grayscale from BGR
            canned = cv2.Canny(grayed,320,320)#For extrating edges we use Canny Edge detection method
            fr.append(frame)#Appending the read frame
            fr_pre.append(canned)#Appending the edge extracted frames
            # cv2_imshow(grayed)#Displaying the original frames
            # cv2_imshow(canned)#Displaying the edge detected frames
            k = cv2.waitKey(10) & 0XFF#this is an arrangement for displaying the video where the secs for which each frame needs to be displayed in given in the paranthesis
            if k == ord('q'):#pressing 'q' key will close the video
                break
        else:
            break
    cap.release()#Here we release the resoures
    cv2.destroyAllWindows()#Here we delete all the windows that were created during the program       
    return fr_pre,fr     #returning the frames received after the execution of function

In [ ]:
def difference_of_frames(frames):
    diff = []#creatin a list variable
    for i in range(0,len(frames)-1):#defining the range
        # print(frames[i],frames[i+1])#checking the frames presence
        diff.append(cv2.absdiff(frames[i],frames[i+1]))#appending the diff between frames to the list variable so we're supposed to get only the difference between frames
    return diff

In [ ]:
def cal_threshold(diff):
  mn = np.mean(diff)#This gives mean
  st_d = np.std(diff)#This gives standard deviation
  a = 4#Setting a random value we can modify it to any value 
  ts = mn + (a * st_d)#defining the standard threshold value for the project/global threshold value
  return ts

In [ ]:
def imp_frames(diff, ts, ogframes):
  a_fr = []#Creating an empty list
  for i in range(len(diff)):#Defining the for loop to be looped over all the frames obtained after finding the frames resulted from subtracting
      mn = np.mean(diff[i])#Calculating the mean for each frame
      st_d = np.std(diff[i])#Calculating the standard deviation for each frame
      fr_ts = mn + (4*st_d)#Finding the threshold values for each frame/image
      a_fr.append([i,fr_ts])#Appending the frame number & the threshold values
  imp_fr = []#Creating an empty list
  for i,ac_tr in(a_fr):#Defining the loop on the list obtained from above code
      if ac_tr >= ts:#Comapring the threshold values to the standard threshold/global threshold values
          imp_fr.append([i,ac_tr])#Appending the list with the imp frames based on their index & the values
  key_fr = []#Creating an empty list
  for i,_ in imp_fr:#Defining the loop over the list obtained from above code
      key_fr.append(ogframes[i])#This extracts the frames based on the index of frames 
  return key_fr

In [ ]:
root_dir="/content/drive/MyDrive/Videos"

In [ ]:
for i, row in df.iterrows():
  if (i>3508):
    path= os.path.join(root_dir,row["Filename"])
    frames,ogframes = framing(path)#calling function framing & then extracting the images 
    diff=difference_of_frames(frames)  
    ts=cal_threshold(diff) 
    key_fr=imp_frames(diff, ts, ogframes)  
    frame_no=key_fr[int(len(key_fr)/2)] #this is a frame
    x=row["Filename"][:-4] 
    cv2.imwrite(f"/content/drive/MyDrive/MainFrames/{x}.jpg",frame_no)
    cv2.destroyAllWindows()
    print(i)